# RAG ChatBot for a Reataurant

This project does this :

1. Answer customer queries about the menu, including dish descriptions, prices, and ingredients.
2. Provide information about dietary options (vegetarian, gluten-free, etc.).
3. Assist with reservations and opening hours.
4. Offer personalized dish recommendations based on customer preferences.
5. Handle frequently asked questions about the restaurant's policies and services.

**Setting up environment**

In [1]:
# installing packages
!pip install -qU langchain-pinecone langchain-text-splitters langchain pinecone-notebooks langchain-community

**Setup for environment variables for Pinecone and Hugging Face API keys**

In [2]:
import os

from google.colab import userdata
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

from google.colab import userdata
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
hf_api_key = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

The document we are using to populate our RAG chatbot application

In [3]:
from langchain_text_splitters import MarkdownHeaderTextSplitter


markdown_document = """
# Bella Vista Italian Restaurant Menu

## Appetizers (Antipasti)

1. Bruschetta al Pomodoro - $12
   Toasted artisanal bread topped with a blend of ripe Roma tomatoes, fresh basil, and extra virgin olive oil. Finished with a drizzle of aged balsamic vinegar.

2. Caprese Salad - $14
   Slices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.

3. Calamari Fritti - $16
   Tender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges.

## Main Courses (Primi Piatti)

4. Spaghetti alla Carbonara - $22
   Al dente spaghetti tossed in a creamy sauce made with farm-fresh eggs, pecorino Romano cheese, and crispy pancetta. Finished with freshly cracked black pepper and parsley.

5. Risotto ai Funghi Porcini - $26
   Arborio rice slowly cooked to creamy perfection with premium dried porcini mushrooms, white wine, and Parmigiano-Reggiano. Garnished with fresh thyme and truffle oil.

6. Lasagna alla Bolognese - $24
   Layers of fresh pasta sheets, rich meat ragù made with ground beef and pork, creamy béchamel sauce, and melted mozzarella. Baked until golden and bubbly.

## Second Courses (Secondi Piatti)

7. Osso Buco alla Milanese - $32
   Slow-braised veal shank in white wine and vegetables, topped with gremolata. Served with saffron risotto and roasted vegetables.

8. Branzino al Forno - $30
   Whole Mediterranean sea bass stuffed with lemon and herbs, oven-roasted and filleted tableside. Served with roasted potatoes and grilled asparagus.

9. Pollo al Marsala - $28
   Tender chicken breast sautéed with mushrooms in a rich Marsala wine sauce. Served with creamy polenta and seasonal vegetables.

## Desserts (Dolci)

10. Tiramisu - $12
    Layers of coffee-soaked ladyfingers and creamy mascarpone cheese, dusted with premium cocoa powder. Served with a shot of espresso on the side.

11. Panna Cotta - $10
    Silky vanilla bean infused cream custard, topped with a mixed berry compote and fresh mint.

12. Cannoli Siciliani - $9
    Two crispy pastry tubes filled with sweet ricotta cream, chocolate chips, and candied orange peel. Dusted with powdered sugar and served with pistachio gelato.

## Beverages (Bevande)

13. Espresso - $3.50
    Single shot of our premium Italian roast coffee

14. Cappuccino - $5
    Espresso topped with steamed milk and foam

15. Italian Sodas - $4.50
    Choice of flavors: Blood Orange, Lemon, or Raspberry

16. House Red Wine (glass) - $9 / (bottle) - $36
    Montepulciano d'Abruzzo - Medium-bodied with notes of cherry and spice

17. House White Wine (glass) - $8 / (bottle) - $32
    Pinot Grigio - Crisp and refreshing with hints of pear and citrus

Our dishes are prepared with the finest ingredients, including imported Italian products and locally sourced organic produce. Gluten-free and vegetarian options are available upon request. Buon appetito!
"""

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

print(md_header_splits)

[Document(metadata={'Header 1': 'Bella Vista Italian Restaurant Menu', 'Header 2': 'Appetizers (Antipasti)'}, page_content='# Bella Vista Italian Restaurant Menu  \n## Appetizers (Antipasti)  \n1. Bruschetta al Pomodoro - $12\nToasted artisanal bread topped with a blend of ripe Roma tomatoes, fresh basil, and extra virgin olive oil. Finished with a drizzle of aged balsamic vinegar.  \n2. Caprese Salad - $14\nSlices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.  \n3. Calamari Fritti - $16\nTender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges.'), Document(metadata={'Header 1': 'Bella Vista Italian Restaurant Menu', 'Header 2': 'Main Courses (Primi Piatti)'}, page_content='## Main Courses (Primi Piatti)  \n4. Spaghetti alla Carbonara - $22\nAl dente spaghetti tossed in a c

Initialising LangChain embedding object

In [4]:
from langchain_pinecone import PineconeEmbeddings

model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=os.environ.get('PINECONE_API_KEY')
)

In [5]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key = pinecone_api_key)

# Setup Index specification to define the cloud service to use and region where we want to deploy our index

from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

# index name :-
index_name = "italian-restaurant"

In [6]:
import time

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.dimension,
        metric="cosine",
        spec=spec
    )
    # wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)



from langchain_pinecone import PineconeVectorStore

namespace = "italian-restaurant"

docsearch = PineconeVectorStore.from_documents(
    documents=md_header_splits,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(1)

In [7]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[2],
        namespace=namespace,
        top_k=1,
        include_values=False,
        include_metadata=True
    )
    print(query)

Feed the Chat Bot using hugging face LLM model


model using here : falcon-7b-instruct ( temp = 0.7 )

In [8]:
from langchain.llms import HuggingFaceHub
from transformers import pipeline
import os
from langchain.chains import RetrievalQA


# Initialize the Hugging Face Hub model
llm = HuggingFaceHub(
    #repo_id="tiiuae/falcon-7b-instruct",  # Replace with any model you prefer, e.g., "EleutherAI/gpt-neo-2.7B"
    repo_id = "meta-llama/Llama-3.2-1B-Instruct",
    model_kwargs={"temperature": 0.7},  # Adjust temperature as needed
    huggingfacehub_api_token=hf_api_key,
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)


<ipython-input-8-99771aa3d26c>:8: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


Write some of the queries to test out the llm training or feeding

In [9]:
query1 = "What are the appetizer options available at Bella Vista Italian Restaurant?"

query2 = "What vegetarian main course options are available, and can you describe one in detail without missing its price?"

query3 = "I want to eat pasta, give me the major ingredients used in the pasta along with it's price?"

In [10]:
all_queries = [query1, query2, query3]

for query in all_queries:
    query_with_knowledge = qa.invoke(query)
    query_without_knowledge = llm.invoke(query)
    print(query_with_knowledge)
    print()
    print(query_without_knowledge)
    print()

{'query': 'What are the appetizer options available at Bella Vista Italian Restaurant?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n# Bella Vista Italian Restaurant Menu  \n## Appetizers (Antipasti)  \n1. Bruschetta al Pomodoro - $12\nToasted artisanal bread topped with a blend of ripe Roma tomatoes, fresh basil, and extra virgin olive oil. Finished with a drizzle of aged balsamic vinegar.  \n2. Caprese Salad - $14\nSlices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.  \n3. Calamari Fritti - $16\nTender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges.\n\n## Beverages (Bevande)  \n13. Espresso - $3.50\nSingle shot of our premium Italian roas

In [11]:
pc.delete_index(index_name)